## 👉 START HERE: How to use this notebook

# Step 3: Build, evaluate, & deploy your Agent

Use this notebook to iterate on the code and configuration of your Agent.

By the end of this notebook, you will have 1+ registered versions of your Agent, each coupled with a detailed quality evaluation.

Optionally, you can deploy a version of your Agent that you can interact with in the [Mosiac AI Playground](https://docs.databricks.com/en/large-language-models/ai-playground.html) and let your business stakeholders who don't have Databricks accounts interact with it & provide feedback in the [Review App](https://docs.databricks.com/en/generative-ai/agent-evaluation/human-evaluation.html#review-app-ui).


For each version of your agent, you will have an MLflow run inside your MLflow experiment that contains:
- Your Agent's code & config
- Evaluation metrics for cost, quality, and latency


**Important note:** Throughout this notebook, we indicate which cell's code you:
- ✅✏️ should customize - these cells contain code & config with business logic that you should edit to meet your requirements & tune quality.
- 🚫✏️ should not customize - these cells contain boilerplate code required to load/save/execute your Agent

*Cells that don't require customization still need to be run!  You CAN change these cells, but if this is the first time using this notebook, we suggest not doing so.*

### 🚫✏️ Install Python libraries

You do not need to modify this cell unless you need additional Python packages in your Agent.

In [0]:
%pip install -qqqq -U -r requirements.txt
# Restart to load the packages into the Python environment
dbutils.library.restartPython()

### 🚫✏️ Connect to Databricks

If running locally in an IDE using Databricks Connect, connect the Spark client & configure MLflow to use Databricks Managed MLflow.  If this running in a Databricks Notebook, these values are already set.

In [1]:
from mlflow.utils import databricks_utils as du

if not du.is_in_databricks_notebook():
    from databricks.connect import DatabricksSession
    import os

    spark = DatabricksSession.builder.getOrCreate()
    os.environ["MLFLOW_TRACKING_URI"] = "databricks"

### 🚫✏️ Load the Agent's UC storage locations; set up MLflow experiment

This notebook uses the UC model, MLflow Experiment, and Evaluation Set that you specified in the [Agent setup](02_agent_setup.ipynb) notebook.

In [2]:
from utils.cookbook.agent_storage_config import AgentStorageConfig
from utils.cookbook.databricks_utils import get_mlflow_experiment_url
from utils.agents.tools import load_obj_from_yaml_file
import mlflow 

# Load the Agent's storage locations
agent_storage_config: AgentStorageConfig= load_obj_from_yaml_file("./configs/agent_storage_config.yaml")

# Show the Agent's storage locations
agent_storage_config.pretty_print()

# set the MLflow experiment
experiment_info = mlflow.set_experiment(agent_storage_config.mlflow_experiment_name)
# If running in a local IDE, set the MLflow experiment name as an environment variable
os.environ["MLFLOW_EXPERIMENT_NAME"] = agent_storage_config.mlflow_experiment_name

print(f"View the MLflow Experiment at {get_mlflow_experiment_url(experiment_info.experiment_id)}")

{
  "uc_model_name": "ep.cookbook_local_test.my_agent",
  "evaluation_set_uc_table": "ep.cookbook_local_test.my_agent_eval_set",
  "mlflow_experiment_name": "/Users/eric.peter@databricks.com/my_agent_mlflow_experiment",
  "class_path": "utils.cookbook.agent_storage_config.AgentStorageConfig"
}
View the MLflow Experiment at https://e2-dogfood.staging.cloud.databricks.com/ml/experiments/3916415516852775


### 🚫✏️ Helper method to log the Agent's code & config to MLflow

Before we start, let's define a helper method to log the Agent's code & config to MLflow.  We will use this to log the agent's code & config to MLflow & the Unity Catalog.  It is used in evaluation & for deploying to Agent Evaluation's [Review App](https://docs.databricks.com/en/generative-ai/agent-evaluation/human-evaluation.html#review-app-ui) (a chat UI for your stakeholders to test this agent) and later, deplying the Agent to production.

In [6]:

import mlflow
from mlflow.types.llm import CHAT_MODEL_INPUT_SCHEMA
from utils.agents.signatures import STRING_RESPONSE_WITH_MESSAGES
from mlflow.models.signature import ModelSignature
from agents.function_calling_agent.function_calling_agent_openai_sdk import FunctionCallingAgent
from utils.agents.function_calling_agent import FunctionCallingAgentConfig

# This helper will log the Agent's code & config to an MLflow run and return the logged model's URI
# If run from inside a mlfow.start_run() block, it will log to that run, otherwise it will log to a new run.
# This logged Agent is ready for deployment, so if you are happy with your evaluation, it is ready to deploy!
def log_agent_to_mlflow(agent_config: FunctionCallingAgentConfig):
    # Get the agent's code path from the imported Agent class
    agent_code_path = f"{os.getcwd()}/{FunctionCallingAgent.__module__.replace('.', '/')}.py"

    # Get the pip requirements from the requirements.txt file
    with open("requirements.txt", "r") as file:
        pip_requirements = [line.strip() for line in file.readlines()] + ["pyspark"] # manually add pyspark

    logged_agent_info = mlflow.pyfunc.log_model(
            artifact_path="agent",
            python_model=agent_code_path,
            input_example=agent_config.input_example,
            model_config=agent_config.model_dump(),
            resources=agent_config.get_resource_dependencies(), # This allows the agents.deploy() command to securely provision credentials for the Agent's databricks resources e.g., vector index, model serving endpoints, etc
            signature=ModelSignature(
            inputs=CHAT_MODEL_INPUT_SCHEMA,
            outputs=STRING_RESPONSE_WITH_MESSAGES #TODO: replace with MLflow signature
        ),
        code_paths=[os.path.join(os.getcwd(), "utils")],
        pip_requirements=pip_requirements,
    )

    return logged_agent_info


## 1️⃣ Iterate on the Agent's code & config to improve quality

The below cells are used to execute your inner dev loop to improve the Agent's quality.

We suggest the following process:
1. Vibe check the Agent for 5 - 10 queries to verify it works
2. Make any necessary changes to the code/config
3. Use Agent Evaluation to evaluate the Agent using your evaluation set, which will provide a quality assessment & identify the root causes of any quality issues
4. Based on that evaluation, make & test changes to the code/config to improve quality
5. 🔁 Repeat steps 3 and 4 until you are satisified with the Agent's quality
6. Deploy the Agent to Agent Evaluation's [Review App](https://docs.databricks.com/en/generative-ai/agent-evaluation/human-evaluation.html#review-app-ui) for pre-production testing
7. Use the following notebooks to review that feedback (optionally adding new records to your evaluation set) & identify any further quality issues
8. 🔁 Repeat steps 3 and 4 to fix any issues identified in step 7
9. Deploy the Agent to a production-ready REST API endpoint (using the same cells in this notebook as step 6)


In [3]:
# Import Cookbook Agent configurations, which are Pydantic models
from utils.agents.function_calling_agent import (
    FunctionCallingAgentConfig,
)
from utils.data_pipeline.data_pipeline_config import (
    DataPipelineConfig,
)
from utils.agents.llm import LLMConfig, LLMParametersConfig
from utils.agents.tools import obj_to_yaml_file, load_obj_from_yaml_file
from utils.agents.vector_search import (
    VectorSearchRetrieverTool,
    VectorSearchSchema,
)
import json

########################
# #### 🚫✏️ Load the Vector Index Unity Cataloglocation from the data pipeline configuration
# Usage:
# - If you used `01_data_pipeline` to create your Vector Index, run this cell.
# - If your Vector Index was created elsewhere, comment out this logic and set the UC location in the Retriever config.
########################

data_pipeline_config: DataPipelineConfig = load_obj_from_yaml_file(
    "./configs/data_pipeline_config.yaml"
)

########################
# #### ✅✏️ Retriever tool that connects to the Vector Search index
########################

retriever_tool = VectorSearchRetrieverTool(
    name="search_product_docs",
    description="Use this tool to search for product documentation.",
    vector_search_index=data_pipeline_config.output.vector_index,
    vector_search_schema=VectorSearchSchema(
        # These columns are the default values used in the `01_data_pipeline` notebook
        # If you used a different column names in that notebook OR you are using a pre-built vector index, update the column names here.
        chunk_text="content_chunked",  # Contains the text of each document chunk
        document_uri="doc_uri",  # The document URI of the chunk e.g., "/Volumes/catalog/schema/volume/file.pdf" - displayed as the document ID in the Review App
        additional_metadata_columns=[],  # Additional columns to return from the vector database and present to the LLM
    ),
    # Optional parameters, see VectorSearchRetrieverTool.__doc__ for details.  The default values are shown below.
    # doc_similarity_threshold=0.0,
    # vector_search_parameters=VectorSearchParameters(
    #     num_results=5,
    #     query_type="ann"
    # ),
    # Adding columns here will allow the Agent's LLM to dynamically apply filters based on the user's query.
    # filterable_columns=[]
)

########################
#### ✅✏️ Agent's LLM configuration
########################

system_prompt = """
## Role
You are a helpful assistant that answers questions using a set of tools. If needed, you ask the user follow-up questions to clarify their request.

## Objective
Your goal is to provide accurate, relevant, and helpful response based solely on the outputs from these tools. You are concise and direct in your responses.

## Instructions
1. **Understand the Query**: Think step by step to analyze the user's question and determine the core need or problem. 

2. **Assess available tools**: Think step by step to consider each available tool and understand their capabilities in the context of the user's query.

3. **Select the appropriate tool(s) OR ask follow up questions**: Based on your understanding of the query and the tool descriptions, decide which tool(s) should be used to generate a response. If you do not have enough information to use the available tools to answer the question, ask the user follow up questions to refine their request.  If you do not have a relevant tool for a question or the outputs of the tools are not helpful, respond with: "I'm sorry, I can't help you with that."
""".strip()

fc_agent_config = FunctionCallingAgentConfig(
    llm_config=LLMConfig(
        llm_endpoint_name="ep-gpt4o-new",  # Model serving endpoint w/ a Chat Completions API
        llm_system_prompt_template=system_prompt,  # System prompt template
        llm_parameters=LLMParametersConfig(
            temperature=0.01, max_tokens=1500
        ),  # LLM parameters
    ),
    # Add one or more tools that comply with the CookbookTool interface
    tools=[retriever_tool],
)

# Print the configuration as a JSON string to see it all together
print(json.dumps(fc_agent_config.model_dump(), indent=4))

########################
##### Dump the configuration to a YAML
# Optional step, this allows the Agent's code file to be run by itself (e.g., outside of this notebook) using the above configuration.
########################
# Import the default YAML config file name from the Agent's code file
from agents.function_calling_agent.function_calling_agent_openai_sdk import FC_AGENT_DEFAULT_YAML_CONFIG_FILE_NAME

# Dump the configuration to a YAML file
obj_to_yaml_file(fc_agent_config, "./configs/"+FC_AGENT_DEFAULT_YAML_CONFIG_FILE_NAME)

{
    "tools": [
        {
            "class_path": "utils.agents.vector_search.VectorSearchRetrieverTool",
            "description": "Use this tool to search for product documentation.",
            "doc_similarity_threshold": 0.0,
            "filterable_columns": [],
            "name": "search_product_docs",
            "retriever_filter_parameter_prompt": "optional filters to apply to the search. An array of objects, each specifying a field name and the filters to apply to that field.",
            "retriever_query_parameter_prompt": "query to look up in retriever",
            "vector_search_index": "ep.cookbook_local_test.product_docs_docs_chunked_index__v1",
            "vector_search_parameters": {
                "num_results": 5,
                "query_type": "ann"
            },
            "vector_search_schema": {
                "additional_metadata_columns": [],
                "chunk_text": "content_chunked",
                "document_uri": "doc_uri"
            }
  

#### ✅✏️ Optionally, adjust the Agent's code

Here, we import the Agent's code so we can run the Agent locally within the notebook.  To modify the code, open the Agent's code file in a separate window, enable reload, make your changes, and re-run this cell.

**Typically, when building the first version of your agent, we suggest first trying to tune the configuration (prompts, etc) to improve quality.  If you need more control to fix quality issues, you can then modify the Agent's code.**

In [ ]:
%load_ext autoreload
%autoreload 2

#### ✅✏️ 🅰 Vibe check the Agent for a single query

Running this cell will produce an MLflow Trace that you can use to see the Agent's outputs and understand the steps it took to produce that output.

If you are running in a local IDE, browse to the MLflow Experiment page to view the Trace (link to the Experiment UI is at the top of this notebook).  If running in a Databricks Notebook, your trace will appear inline below.

In [ ]:
from utils.cookbook.databricks_utils import get_mlflow_experiment_traces_url
from agents.function_calling_agent.function_calling_agent_openai_sdk import FunctionCallingAgent

# Load the Agent's code with the above configuration
agent = FunctionCallingAgent(fc_agent_config)

# Vibe check the Agent for a single query
output = agent.predict(model_input={"messages": [{"role": "user", "content": "How does the blender work?"}]})

print(f"View the MLflow Traces at {get_mlflow_experiment_traces_url(experiment_info.experiment_id)}")
print(f"Agent's final response:\n----\n{output['content']}\n----")
print()
print(f"Agent's full message history (useful for debugging):\n----\n{json.dumps(output['messages'], indent=2)}\n----")


Now, let's test a multi-turn conversation with the Agent.

In [ ]:
second_turn = {'messages': output['messages'] + [{"role": "user", "content": "How do I turn it on?"}]}

# Run the Agent again with the same input to continue the conversation
second_turn_output = agent.predict(model_input=second_turn)

print(f"View the MLflow Traces at {get_mlflow_experiment_traces_url(experiment_info.experiment_id)}")
print(f"Agent's final response:\n----\n{second_turn_output['content']}\n----")
print()
print(f"Agent's full message history (useful for debugging):\n----\n{json.dumps(second_turn_output['messages'], indent=2)}\n----")

#### ✅✏️ 🅱 Evaluate the Agent using your evaluation set

Note: If you do not have an evaluation set, you can create a synthetic evaluation set by using the 03_synthetic_evaluation notebook.

In [ ]:
evaluation_set = spark.table(agent_storage_config.evaluation_set_uc_table)

with mlflow.start_run():
    logged_agent_info = log_agent_to_mlflow(fc_agent_config)

    # Run the agent for these queries, using Agent evaluation to parallelize the calls
    eval_results = mlflow.evaluate(
        model=logged_agent_info.model_uri,  # use the MLflow logged Agent
        data=evaluation_set,  # Evaluate the Agent for every row of the evaluation set
        model_type="databricks-agent",  # use Agent Evaluation
    )

    # Show all outputs.  Click on a row in this table to display the MLflow Trace.
    display(eval_results.tables["eval_results"])

    # Click 'View Evaluation Results' to see the Agent's inputs/outputs + quality evaluation displayed in a UI

## 2️⃣ Deploy a version of your Agent - either to the Review App or Production

Once you have a version of your Agent that has sufficient quality, you will register the Agent's model from the MLflow Experiment into the Unity Catalog & use Agent Framework's `agents.deploy(...)` command to deploy it.  Note these steps are the same for deploying to pre-production (e.g., the [Review App](https://docs.databricks.com/en/generative-ai/agent-evaluation/human-evaluation.html#review-app-ui) or production.

By the end of this step, you will have deployed a version of your Agent that you can interact with and share with your business stakeholders for feedback, even if they don't have access to your Databricks workspace:

1. A production-ready scalable REST API deployed as a Model Serving endpoint that logged every request/request/MLflow Trace to a Delta Table.
    - REST API for querying the Agent
    - REST API for sending user feedback from your UI to the Agent
2. Agent Evaluation's [Review App](https://docs.databricks.com/en/generative-ai/agent-evaluation/human-evaluation.html#review-app-ui) connected to these endpoints.
3. [Mosiac AI Playground](https://docs.databricks.com/en/large-language-models/ai-playground.html) connected to these endpoints.

Option 1: Deploy the last agent you logged above

In [ ]:
from databricks import agents

# Use Unity Catalog as the model registry
mlflow.set_registry_uri("databricks-uc")

# Register the Agent's model to the Unity Catalog
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=agent_storage_config.uc_model_name
)

# Deploy the model to the review app and a model serving endpoint
agents.deploy(agent_storage_config.uc_model_name, uc_registered_model_info.version)

Option 2: Log the latest copy of the Agent's code/config and deploy it

In [8]:
from databricks import agents

# Use Unity Catalog as the model registry
mlflow.set_registry_uri("databricks-uc")

with mlflow.start_run():
    logged_agent_info = log_agent_to_mlflow(fc_agent_config)

    # Register the Agent's model to the Unity Catalog
    uc_registered_model_info = mlflow.register_model(
        model_uri=logged_agent_info.model_uri, name=agent_storage_config.uc_model_name
    )

# Deploy the model to the review app and a model serving endpoint
agents.deploy(agent_storage_config.uc_model_name, uc_registered_model_info.version)

Uploading artifacts:   0%|          | 0/43 [00:00<?, ?it/s]

Registered model 'ep.cookbook_local_test.my_agent' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/43 [00:00<?, ?it/s]

Created version '2' of model 'ep.cookbook_local_test.my_agent'.
2024/11/06 13:41:23 INFO mlflow.tracking._tracking_service.client: 🏃 View run angry-cub-872 at: https://e2-dogfood.staging.cloud.databricks.com/ml/experiments/3916415516852775/runs/113eadcfe9994f528515fad03140eec4.
2024/11/06 13:41:23 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://e2-dogfood.staging.cloud.databricks.com/ml/experiments/3916415516852775.


BadRequest: Could not parse request object: Expected both 'key' and 'value' to be set on elements in the field 'environment_vars'
 at [Source: (ByteArrayInputStream); line: 1, column: 396]
 at [Source: java.io.ByteArrayInputStream@9a6a427; line: 1, column: 396]